In [1]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

In [3]:
# the data, split between train and test sets
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [5]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])



In [6]:
import time
start_time = time.time()
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 7s 110us/step - loss: 0.2589 - accuracy: 0.9201 - val_loss: 0.0552 - val_accuracy: 0.9824
Epoch 2/12
60000/60000 [==============================] - 4s 71us/step - loss: 0.0882 - accuracy: 0.9729 - val_loss: 0.0405 - val_accuracy: 0.9867
Epoch 3/12
60000/60000 [==============================] - 4s 72us/step - loss: 0.0669 - accuracy: 0.9801 - val_loss: 0.0376 - val_accuracy: 0.9877
Epoch 4/12
60000/60000 [==============================] - 4s 71us/step - loss: 0.0579 - accuracy: 0.9829 - val_loss: 0.0342 - val_accuracy: 0.9878
Epoch 5/12
60000/60000 [==============================] - 4s 71us/step - loss: 0.0477 - accuracy: 0.9855 - val_loss: 0.0319 - val_accuracy: 0.9887
Epoch 6/12
60000/60000 [==============================] - 4s 71us/step - loss: 0.0428 - accuracy: 0.9871 - val_loss: 0.0310 - val_accuracy: 0.9893
Epoch 7/12
60000/60000 [==============================] - 4s 71us/s

In [7]:
print("--- %s seconds to train ---" % (time.time() - start_time))

--- 54.33684778213501 seconds to train ---


In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.029053883073842553
Test accuracy: 0.9915000200271606
